<img src=https://upload.wikimedia.org/wikipedia/commons/6/69/Logo_Austral_Negativo_Vertical.jpg width="250"/> 

In [76]:
import requests
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
import warnings
import urllib.request
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.options import Options
import time
from datetime import datetime
import csv
import fileinput
warnings.filterwarnings("ignore")

In [77]:
url = "https://contenidosweb.prefecturanaval.gob.ar/alturas/"
html = requests.get(url).content
soup = BeautifulSoup(html)

In [78]:
time.sleep(3)

In [79]:
gdp = soup.find_all("table", attrs={"class": "table table-hover fpTable"})

In [80]:
table1 = gdp[0]
body = table1.find_all("tr")
head = body[0]
body_rows = body[1:]
headings = []
for item in head.find_all("th"):
    item = (item.text).rstrip("\n")
    headings.append(item)
del headings[0]
print(headings)


['Río', 'Ult. registro', 'Variación', 'Periodo', 'Fecha Hora', 'Estado', '', 'Registro Anterior', 'Fecha Anterior', 'Alerta', 'Evacuación', 'Hist.']


In [81]:
all_rows1 = [] 
for row_num in range(len(body_rows)):
    row = [] 
    for row_item in body_rows[row_num].find_all("th"):
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        row.append(aa)
    all_rows1.append(row)    

In [82]:
df1 = pd.DataFrame(data=all_rows1,columns=["Puerto"])

In [83]:
all_rows2 = [] 
for row_num in range(len(body_rows)):
    row = []
    for row_item in body_rows[row_num].find_all("td"):
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        row.append(aa)
    all_rows2.append(row)

In [84]:
df2 = pd.DataFrame(data=all_rows2,columns=headings)

In [85]:
df = pd.concat([df1, df2], axis=1, join='inner')

In [86]:
df["Ult. registro"] = pd.to_numeric(df["Ult. registro"], errors='coerce').fillna(0, downcast='infer')
df["Variación"] = pd.to_numeric(df["Variación"], errors='coerce').fillna(0, downcast='infer')
df["Periodo"] = pd.to_numeric(df["Periodo"], errors='coerce').fillna(0, downcast='infer')
df["Registro Anterior"] = pd.to_numeric(df["Registro Anterior"], errors='coerce').fillna(0, downcast='infer')
df = df.astype({"Estado":'category'})
#df["Fecha Hora"] = pd.to_datetime(df["Fecha Hora"])
df["Fecha Anterior"] = pd.to_datetime(df["Fecha Anterior"])

In [87]:
df.set_index("Fecha Hora", inplace = True)

In [88]:
#save to CSV
#no tiene headings: el archivo creado tiene que tenerlo
df.to_csv('prefectura.csv', mode='a', index=["Fecha Hora"], header=False)

In [89]:
#saco duplicados
inFile = open('prefectura.csv','r')

outFile = open('df.csv','w')

listLines = []

for line in inFile:

    if line in listLines:
        continue

    else:
        outFile.write(line)
        listLines.append(line)

outFile.close()

inFile.close()

pasar Neural prophet para forecastear altura de rio en un punto x.
